In [63]:
#install dependencies
!pip install spotipy --upgrade 
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Requirement already up-to-date: spotipy in /usr/local/lib/python3.6/dist-packages (2.12.0)
Requirement already up-to-date: google-api-python-client in /usr/local/lib/python3.6/dist-packages (1.8.4)
Requirement already up-to-date: google-auth-httplib2 in /usr/local/lib/python3.6/dist-packages (0.0.3)
Requirement already up-to-date: google-auth-oauthlib in /usr/local/lib/python3.6/dist-packages (0.4.1)


In [0]:
#imports for API request
import json
import spotipy as sp
from spotipy.oauth2 import SpotifyClientCredentials
import csv

In [0]:
#imports for NN
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook as tqdm
from torch import utils

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
#Get authorization
token = sp.oauth2.SpotifyClientCredentials(client_id='0ae72c9b0d4948d5ba0206c4c350d3fe', client_secret='c1db0c06c2df46b7b9219250080adc8d')
cache_token = token.get_access_token()
spotify = sp.Spotify(cache_token)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [68]:
spotify.user_playlist_tracks('pkabranov', '4Kvvbir7qtwhF5xIeRILOl', limit=15, offset=0).get('total')

1222

In [0]:
#Get IDs in specified playlist 
import pandas as pd
def get_playlist_ids(playlist_id):
  offset = 0
  track_ids = []
  while offset < spotify.user_playlist_tracks('pkabranov', playlist_id, offset=0).get('total'):
    #print(offset)
    results = spotifey.user_playlist_tracks('pkabranov', playlist_id, limit=1, offset=offset)
    for item in results['items']:
      if not item in track_ids: 
        track_ids.append(item['track']['id'])
    offset += 1
  return track_ids
rishi_ids = get_playlist_ids('4Kvvbir7qtwhF5xIeRILOl')

In [0]:
#Get usable features for each track in track_ids

def get_playlist_features(track_ids):
  batch, interval = 0, 100
  all_usable_track_features = []

  while batch < len(track_ids) - interval:
    all_usable_track_features.extend(spotify.audio_features(track_ids[batch:(batch + interval)]))
    batch += interval
    if batch > len(track_ids) - interval:
      all_usable_track_features.extend(spotify.audio_features(track_ids[batch:(batch + len(track_ids) % interval)]))

  usable_keys = ['duration_ms', 'id', 'key', 'loudness', 'tempo', 'time_signature']
  usable_track_features = []

  for track in all_usable_track_features:
    if track:
      usable_track_features.append({key: value for (key, value) in track.items() if key in usable_keys})

  return usable_track_features

In [80]:
def df_gen(playlist_id):
    ids = get_playlist_ids(playlist_id)
    feat = get_playlist_features(ids)
    id_df = pd.DataFrame.from_dict(ids, orient = "columns")
    feat_df = pd.DataFrame.from_dict(feat, orient = "columns")
    df = id_df.join(feat_df)
    return df
  
features = df_gen('4Kvvbir7qtwhF5xIeRILOl')
features

,0,key,loudness,tempo,id,duration_ms,time_signature
0,3IvMYBE7A3c7to1aEcfFJk,7,-3.680,77.888,3IvMYBE7A3c7to1aEcfFJk,222929,4
1,5TCBWmEBrin7etRa4Lswr1,1,-6.808,170.718,5TCBWmEBrin7etRa4Lswr1,223124,4
2,4wVOKKEHUJxHCFFNUWDn0B,10,-7.507,122.947,4wVOKKEHUJxHCFFNUWDn0B,220488,4
3,3Q4gttWQ6hxqWOa3tHoTNi,9,-9.299,86.318,3Q4gttWQ6hxqWOa3tHoTNi,269680,4
4,466cKvZn1j45IpxDdYZqdA,1,-8.820,81.604,466cKvZn1j45IpxDdYZqdA,247059,4
...,...,...,...,...,...,...,...
1217,3oDkdAySo1VQQG0ptV7uwa,1,-5.515,163.935,3oDkdAySo1VQQG0ptV7uwa,175755,4
1218,6cjSh516BbrNUs213jDAGm,0,-5.286,154.968,6cjSh516BbrNUs213jDAGm,177847,4
1219,6Zc5f20q7sOe1IC1c1jQvZ,6,-6.103,136.020,6Zc5f20q7sOe1IC1c1jQvZ,228816,4
1220,7vbjH8laThXpcBXeLNPm3M,2,-7.635,153.028,7vbjH8laThXpcBXeLNPm3M,189800,4


In [0]:
import urllib
from tqdm import tqdm
import numpy as np

In [87]:
def get_artist_song_ids():
  with open('drive/My Drive/predictify/data_table_2.csv', newline='') as csvfile:
      spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
      #row_count = sum([1 for row in spamreader])

      all_rows_df = pd.DataFrame(columns = ['artist', 'song', 'id', 'play count'])

      for row in spamreader:
          url = urllib.parse.unquote(row[1] + "%20artist:" + row[0])
          search_result = spotify.search(url, type='track', limit=1)
          try:
            curr_row_df = pd.DataFrame({'artist': [search_result.get('tracks')['items'][0]['artists'][0].get('name')], #artist name
                                        'song': [search_result.get('tracks')['items'][0]['name']], #song name
                                        'id': [search_result.get('tracks')['items'][0]['id']], #song id
                                        'play count': row[2]})
            all_rows_df = all_rows_df.append(curr_row_df)
          except:
            continue          

      return all_rows_df
          
from_csv = get_artist_song_ids()
from_csv

,artist,song,id,play count
0,Drake,Deep Pockets,3IvMYBE7A3c7to1aEcfFJk,12482146
0,Drake,When To Say When,5TCBWmEBrin7etRa4Lswr1,12885199
0,Drake,Chicago Freestyle (feat. Giveon),4wVOKKEHUJxHCFFNUWDn0B,36258296
0,Drake,Not You Too (feat. Chris Brown),3Q4gttWQ6hxqWOa3tHoTNi,15311250
0,Drake,Toosie Slide,127QTOFJsJQp5LbJbu3A1y,273281855
...,...,...,...,...
0,Lil Baby,Ride Or Die,5N6z4rWwioNXS6kxEsUhVW,1625912
0,Lil Baby,Stendo,7GSlWq7g7RHCbsPMrJ0tEx,4725938
0,Lil Baby,Large,0NhuiLOx2ptpPIhAERs33C,1173485
0,Lil Baby,Pink Slip,4cAIfmo7P8InMxyyXsZtoM,2637854


In [107]:
#add play count to features dataframe
joined = pd.merge(from_csv, features, on='id')
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

joined.drop(0, axis=1)

,artist,song,id,play count,key,loudness,tempo,duration_ms,time_signature
0,Drake,Deep Pockets,3IvMYBE7A3c7to1aEcfFJk,12482146,7,-3.680,77.888,222929,4
1,Drake,When To Say When,5TCBWmEBrin7etRa4Lswr1,12885199,1,-6.808,170.718,223124,4
2,Drake,Chicago Freestyle (feat. Giveon),4wVOKKEHUJxHCFFNUWDn0B,36258296,10,-7.507,122.947,220488,4
3,Drake,Not You Too (feat. Chris Brown),3Q4gttWQ6hxqWOa3tHoTNi,15311250,9,-9.299,86.318,269680,4
4,Drake,Toosie Slide,127QTOFJsJQp5LbJbu3A1y,273281855,1,-9.750,81.618,247059,4
...,...,...,...,...,...,...,...,...,...
811,Lil Baby,My Jeans (Lil Baby & Gunna feat. Young Thug),4T7cJdrTRqOBESPYfvfj2x,13100384,11,-5.869,100.576,196448,5
812,Lil Baby,Close Friends (Lil Baby & Gunna),5nzhL67GpF9szXWg5y509A,182748232,10,-4.669,158.896,203881,4
813,Lil Baby,Drip Too Hard (Lil Baby & Gunna),78QR3Wp35dqAhFEc2qAGjE,622634854,1,-6.903,112.511,145543,4
814,Lil Baby,Drip Too Hard (Lil Baby & Gunna),78QR3Wp35dqAhFEc2qAGjE,622634854,1,-6.903,112.511,145543,4


In [0]:
#Import and format data

Dataloader()

In [0]:
#Define architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.advance = nn.Sequential(nn.Linear(5, 5),
                                     nn.ReLU(),
                                     nn.Linear(5, 5),
                                     nn.ReLU(),
                                     nn.Linear(5, 5),
                                     nn.Softmax()
                                    )
        print(self)

    def forward(self, features):
        output = self.advance(features.float())
        return output

net = Net()

Net(
  (advance): Sequential(
    (0): Linear(in_features=5, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=5, bias=True)
    (5): Softmax(dim=None)
  )
)


In [0]:
import tensorflow as tf
import functools

import pandas as pd
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model

Load the data from csv file URLS and visualize:

In [0]:
TRAIN_DATA_URL = ".csv" #.csv file url for train data
TEST_DATA_URL = ".csv" #csv file url for test data

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("test.csv", TEST_DATA_URL)

LABEL_COLUMN = 'popularity_score' #the column name of the value the model predicts
LABELS = [0, 1]

#load dataset
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size = , #insert batch size
        label_name = LABEL_COLUMN,
        num_epochs = 1,
        ignore_errors = True,
        **kwargs)
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path) 

#visualize dataset
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:10s}: {}".format(key,value.numpy()))

Organize the data into continuous and categorical data (i dont think we have cat data) that cna be analyzed #assume that the csv file has the labels already on the first row, otherwise pass it in 
`make_csv_dataset`



In [0]:
SELECT_COLUMNS = ['duration_ms', 'key', 'loudness', 'tempo', 'time_signature'] #omit the ID column
DEFAULTS = [0.0, 0, 0.0, 0.0, 0]

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS, column_defaults = DEFAULTS)
show_batch(temp_dataset) #visualization, can remove

#pack the numerical data into vectors prior to preprocessing
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

NUMERIC_FEATURES = SELECT_COLUMNS

packed_train_data = raw_train_data.map(PackNumericFeatures(NUMERIC_FEATURES))
packed_test_data = raw_test_data.map(PackNumericFeatures(NUMERIC_FEATURES))

Perform some sort of data preprocessing on the continuous data (something basic rn like normalization or mean subtraction) 

Time signature and key are categorical data. Key ranges from -1 to 12 in the following manner:

Key: -1: No key 
     0: C major
     1: C# major

**ISSUE:** All values of time signature aren't known



In [0]:
#Continuous data
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc #visualize our song data in pandas table

MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_columns

#Categorical data 
CATEGORIES = {
    'key': ['-1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'],
    'time_signature' : ['3', '4'],
}
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

Soem layers for use in the net with the preprocessed data


In [0]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
tf.keras.layers.Dense(1) 

Downloading spotify songs from a playlist as MP3s for potential waveform analysis 

I think that the best way to store data would be to write it into a google sheet that's easily accessible so I'm trying to do that here

TO-DO: 

1.   put the credentials.json file into the path (wehre tf is the path for google drive)
2.   make sure dataframes are the correct types (im not sure what the spotify.user_playlist_tracks return type is, but format the pandas df into a convenient datatype) -- like can i put the return of this_iter_track_ids into a pd dataframe without any dtype conversions smoothly?
3. make sure the update_cells usage is correct for pygsheets
3.   get_track_features method



In [0]:
import pygsheets
import pandas as pd

'''
Access tokens here
'''

USERNAME = ''
PLAYLIST_ID = ''

#auth
gc = pygsheets.authorize(service_file='drive/My Drive/predictify/predictify-data-a664cf501a42.json') #fill in the credentials.json here

#open the sheet
sh = gc.open('Song Data')
wks = sh[0]
#method that stores the data as a pandas dataframe and then shoves the dataframe into the google sheets
def get_track_ids(username, playlist_id):
  OFFSET = 0
  track_ids = pd.Dataframe() #dataframe init
  for i in range(spotify.user_playlist_tracks(USERNAME, PLAYLIST_ID, fields = total): #for the number of tracks in the playlist ID, offset increases by 100 every time
    this_iter_track_ids = spotify.user_playlist_tracks(USERNAME, PLAYLIST_ID, limit = 100, OFFSET)
    track_ids = track_ids.append(this_iter_track_ids)
    OFFSET += 100
  track_ids = pd.concat(track_ids, ignore_index = True)
  #update the first sheet with df, starting at cell A2.
  return track_ids

#extract the track ids from either the initial pd dataframe or the google sheet and use it to get song features, then store the features into df and google sheets
def get_track_features(username, track_ids):
  track_features = pd.Dataframe() #dataframe init
  keys = ['duration_ms', 'id', 'key', 'loudness', 'tempo', 'time_signature']
  for i in range(len(get_track_ids(USERNAME, PLAYLIST_ID).index)):
    #append the data into track_features
    this_iter_track_features = track_features.append({key: value for (key, value in track.items() if key in keys)})
  return track_features

column_data = [get_track_ids(), get_track_features()]
complete_data = pd.concat(column_data)
wks.set_dataframe(complete_data, (2, 1), extend = True, escape_formulae = True)
wks.export(filetype = 'csv', filename = 'SongData')